In [1]:
import os
import shutil

for root, dirs, files in os.walk('/work'):
    for f in files:
        os.unlink(os.path.join(root, f))
    for d in dirs:
        shutil.rmtree(os.path.join(root, d))

In [2]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]
!pip install anvil-uplink
!pip install --upgrade pip
!pip3 install --upgrade protobuf==3.20.0

  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Not uninstalling idna at /shared-libs/python3.7/py-core/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'idna'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.6.0, but you have ipykernel 6.16.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.
google-colab 1.0.0 requires notebook~=5.2.0, but you have notebook 6.5.1 which is incompatible.
google-colab 1.0.0 requires six~=1.12.0, but you have six 1.16.0 which is incompatible.
google-colab 1.0.0 requires tornado~=4.5.0, but you have tornado 6.2 which is incompatible.
tensorboard 2.10.1 requires google-auth<3,>=1.6.3

In [3]:
!wget -q http://ftp.de.debian.org/debian/pool/main/i/icu/libicu63_63.1-6+deb10u3_amd64.deb
!dpkg -i /work/libicu63_63.1-6+deb10u3_amd64.deb
!sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys EB3E94ADBE1229CF 
!wget -q https://dotnet.microsoft.com/download/dotnet/scripts/v1/dotnet-install.ps1
!wget -q https://packages.microsoft.com/config/ubuntu/18.04/prod.list
!sudo mv prod.list /etc/apt/sources.list.d/microsoft-prod.list
!sudo apt update
!sudo apt-get install libicu63

(Reading database ... 33281 files and directories currently installed.)
Preparing to unpack .../libicu63_63.1-6+deb10u3_amd64.deb ...
Unpacking libicu63:amd64 (63.1-6+deb10u3) over (63.1-6+deb10u3) ...
Setting up libicu63:amd64 (63.1-6+deb10u3) ...
Processing triggers for libc-bin (2.28-10+deb10u2) ...
Executing: /tmp/apt-key-gpghome.bDpsA8yo0i/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys EB3E94ADBE1229CF
gpg: key EB3E94ADBE1229CF: "Microsoft (Release signing) <gpgsecurity@microsoft.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Hit:1 http://deb.debian.org/debian buster InRelease
Get:2 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Hit:4 https://packages.microsoft.com/ubuntu/18.04/prod bionic InRelease
Fetched 91.5 kB in 0s (262 kB/s)



9 packages can be upgraded. Run 'apt list --upgradable' to see them.



libicu63 is already the newest version (63

In [4]:
!sudo apt-get install aspnetcore-runtime-2.2 -y 
!sudo apt-get install dotnet-sdk-2.2 -y
!git clone https://github.com/microsoft/verisol.git
!cd /work/verisol/Sources
!dotnet build /work/verisol/Sources




aspnetcore-runtime-2.2 is already the newest version (2.2.8-1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.



dotnet-sdk-2.2 is already the newest version (2.2.402-1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Cloning into 'verisol'...
remote: Enumerating objects: 5429, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 5429 (delta 37), reused 113 (delta 27), pack-reused 5276
Receiving objects: 100% (5429/5429), 3.46 MiB | 39.34 MiB/s, done.
Resolving deltas: 100% (3802/3802), done.
Checking out files: 100% (584/584), done.



==  Restore completed in 153.34 ms for /work/verisol/Sources/BoogieAST/BoogieAST.csproj.
  Restore completed in 41.34 ms for /work/verisol/Sources/ExternalToolsManager/ExternalToolsManager.csproj.
  Restore completed in 22.62 ms for /work/verisol/Sources/SolToBoogie/SolToBoogie.csproj.
  Restore completed in 17.39 ms for /work/verisol/Sources/SolToBoo

In [5]:
import anvil.server
anvil.server.connect("server_ZFIQMJB4URWM7V75MDXTD4R5-FNQ76OID3FCLIPND")
import os
import os.path
from os import path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rich.table import Column, Table
from rich import box
from rich.console import Console

'''
this is for debuggin purpose of the pipeline
it should be supplied by user from the app 
'''
CONTRACT_PATH = ''
user_params = {
      '1': '/content/drive/MyDrive/model_files',
      '2': '/content/drive/MyDrive/LoopFor.sol,LoopFor',
      '3': '/content/drive/MyDrive/pred.txt',
      '4': '/content/drive/MyDrive/sample.txt'
      }
test_params = {
      'batch_size': 8,
      'shuffle': False,
      'num_workers': 0
      }

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
2023-02-26 20:36:13.423844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 20:36:13.551967: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 20:36:13.556754: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-2

In [6]:
class TestDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, source_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.source_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length=self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                              padding="max_length",\
                                              return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
    }

In [7]:
#test how the model performs on never-seen test contracts

@anvil.server.callable
def predict_contract(saved_model_path, test_contract_path, prediction_file):
   #in test_contract_path, user should specify path, contract name arguments
   global CONTRACT_PATH 
   CONTRACT_PATH = test_contract_path
   contracts = test_contract_path.split(',') 
   if saved_model_path == 'no':
      model = T5ForConditionalGeneration.from_pretrained('sallywww/invariants-model')
   else:
      model = T5ForConditionalGeneration.from_pretrained(saved_model_path)  
   tokenizer = T5Tokenizer.from_pretrained("t5-small")
   col = ['Test']
   f = open(contracts[0], "r")
   file = f.read()
   df = pd.DataFrame([file], columns=col)
   col = ['Test']
   df = pd.DataFrame([file], columns=col)
   model.eval()
   test_val = TestDataSetClass(df, tokenizer, source_len=512, source_text="Test")
   test_loader = DataLoader(test_val, **test_params)
   predictions = []
   with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
          ids = data['source_ids']
          mask = data['source_mask']
          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          predictions.extend(preds)
   if prediction_file != 'no': 
      with open(prediction_file, 'w') as writefile:
          for line in predictions:
            writefile.write(line + '\n')
   with open('contract.sol', 'w') as writefile:
      for line in predictions:
        writefile.write(line + '\n')
   return predictions

In [8]:
@anvil.server.callable
def run_verisol(verify_original_contract, verisol_flags, \
                verification_result_path):
  if path.exists('/work/boogie.txt'):
      os.remove('/work/boogie.txt')
  if path.exists('/work/corral.txt'):
      os.remove('/work/corral.txt')
  boogie_file = []
  if verify_original_contract == 'yes':
    contracts = CONTRACT_PATH.split(',')
    print(contracts)
    var1 = contracts[0]
    var2 = contracts[1]
    var3 = verisol_flags
    !dotnet /work/verisol/bin/Debug/VeriSol.dll $'{var1}' $'{var2}' $'{var3}'
    with open('/work/boogie.txt', 'r') as boogief: 
        for line in boogief:
          boogie_file.extend([line])
        if verification_result_path != None: 
          with open(verification_result_path, 'w') as writefile:
            for line in boogief:
              writefile.write(line +'\n')
        if path.exists('/work/corral.txt'):
          with open('/work/corral.txt', 'r') as corraltxt: 
            boogie_file.extend(['Proof not found! Below is bounded model checking results from Corral:\n'])
            for line in corraltxt:            
              boogie_file.extend([line])
            
  else:
    var1 ='/work/contract.sol'
    var2 = contracts[1]
    var3 = verisol_flags
    !dotnet /work/verisol/bin/Debug/VeriSol.dll $'{var1}' $'{var2}' $'{var3}'
    with open('/work/boogie.txt', 'r') as boogief: 
        for line in boogief:
          boogie_file.extend([line])
        if verification_result_path != 'no': 
          with open(verification_result_path, 'w') as writefile:
            for line in boogief:
              writefile.write(line +'\n')
        if path.exists('/work/corral.txt'):
          with open('/work/corral.txt', 'r') as corraltxt: 
            boogie_file.extend(['Proof not found! Below is bounded model checking results from Corral:\n'])
            for line in corraltxt:            
              boogie_file.extend([line])

  return boogie_file

In [8]:
anvil.server.wait_forever()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d0de821e-6b37-422f-9bbc-a2df12491510' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>